# CDC County Data Transformation

* **This data will be explored for cleaning**
* Data will be used to join with other social and medical information

Note, this notebook is especially long, because values were being checked to see what types of null substitutions to perform, etc.  

**First perform imports**

In [1]:
import pandas as pd
# ref https://stackoverflow.com/questions/23045318/scikit-grid-search-over-multiple-classifiers

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MaxAbsScaler,FunctionTransformer
from sklearn.svm import SVR, SVC
from sklearn.ensemble import GradientBoostingClassifier, \
                            AdaBoostClassifier,AdaBoostRegressor, \
                            VotingClassifier, \
                            RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor # can use KNeighborsClassifier as regressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import datasets
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LinearRegression

import pickle
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import datetime as datetime


**Load extracted CDC county csv data into pandas**

In [2]:
counties_df = pd.read_csv('../data/cdc_counties.tmp.csv')

/Users/jenfong/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Preliminary EDA - null counts

There are a lot of columns with nulls and are hard to see in the notebook.  Here we take an initial look at columns with literals to get an idea of the extent of the nulls

In [4]:

counties_df['CBSA type'].isna().sum()


168095

In [5]:
 
counties_df['County type'].isna().sum()


16071

In [6]:
counties_df['IHE with >5000 full-time enrollment'].isnull().sum() 

6399

#### Check nulls and value range

In [7]:
counties_df['Population as a percent of CBSA'].isnull().sum()

168095

Check values to make sure that 1.0 means 100%

In [8]:
counties_df['Population as a percent of CBSA'].describe()

count    237584.000000
mean          0.490084
std           0.416490
min           0.001200
25%           0.079625
50%           0.347950
75%           1.000000
max           1.000000
Name: Population as a percent of CBSA, dtype: float64

**Identify columns which contribute data points that can vary from county to county and do not give a sense of a change in covid trajectory.  Most of these columns have similar columns in the dataframe that show change**

This is a first pass

In [9]:
drop_columns = ['IHE Full-time enrollment', 'CBSA', 'Population as a percent of CBSA', 'FEMA region',
               'Cases - last 7 days', 'Cases per 100k - last 7 days','Deaths - last 7 days','Deaths per 100k - last 7 days',
               'Cases - previous 7 days','Cases per 100k - previous 7 days','Deaths - previous 7 days',
                'Deaths per 100k - previous 7 days','Cumulative cases','Cumulative cases per 100k','Cumulative deaths',
                'Cumulative deaths per 100k','Total RT-PCR diagnostic tests - last 7 days (may be an underestimate due to delayed reporting)',
                'RT-PCR tests per 100k - last 7 days (may be an underestimate due to delayed reporting)','Median test latency - last 7 days',
                'Viral (RT-PCR) lab test positivity rate - previous 7 days (may be an underestimate due to delayed reporting)','Total RT-PCR diagnostic tests - previous 7 days (may be an underestimate due to delayed reporting)',
                'RT-PCR tests per 100k - previous 7 days(may be an underestimate due to delayed reporting)','Median test latency - last 7 days.1',
                '% tests resulted in 3 or fewer days - last 7 days.1']

In [10]:
reduced_df = counties_df.drop(columns=drop_columns)

Second pass at dropping data

In [11]:
drop_columns_2 = ['Total # of hospital CCNs','Total inpatient beds among hospitals reporting - last 7 days',
                  'Total staffed adult ICU beds among hospitals reporting - last 7 days',
                  'Total ventilators among hospitals reporting - last 7 days','People who are fully vaccinated',
                 'People who are fully vaccinated - ages 65+', 'Population']

In [12]:
reduced_df.drop(columns = drop_columns_2, inplace=True)

We make our third pass at identifying dropping candidates, columns with literals or values which won't be used in visualization or modeling.  'County only' was a column that was created through feature engineering; however many states have counties with the same name, so this hasn't been helpful for joining or checking differences

In [13]:
drop_columns_3 = ['County only', 'Median test latency - 15-21 days ago',
                  'Total RT-PCR diagnostic tests - 15-21 days ago (may be an underestimate due to delayed reporting)',
                  'RT-PCR tests per 100k - 15-21 days ago (may be an underestimate due to delayed reporting)']

In [14]:
reduced_df.drop(columns=drop_columns_3, inplace=True)

## See what labels can be mapped to ordinals in a possible sorted order

Random Forest breaks up the data by sorting at each node.  Most models don't handle labels well.  Here we look to see if the State Abbreviations can be assigned numbers and a sorted order mapped to these states, rather than doing one hot encoding.  

In the end, we decided to leave in the State Abbreviations for possible use in the visualizations from all our combined datasets, while not using it directly in modeling.

In [15]:
counties_df[(counties_df['Forecasted case trajectory'] == 'Likely Decreasing')][['Forecasted case trajectory', 'State Abbreviation']].\
groupby('State Abbreviation').agg('count').sort_values(by='Forecasted case trajectory', ascending=False) #.index

,Forecasted case trajectory
State Abbreviation,
GA,2746
TX,2280
NC,1873
VA,1825
TN,1761
IN,1728
KY,1656
PA,1613
OH,1606


Given our target column, the 'Forecasted Case Trajectory', we determined that the model wouldn't be able to do much with each states population or case number data, since the context determines how much the number means and that varies from state to state.  (We also ran diagnostic models using this data and accuracy scores were low)  

The final determination was to drop these columns and keep those columns which kept more contextualized information that showed the rate of change from the last worksheet to the current one.

The drops were done in stages

We look at the IHE Column to check values and to see if the nulls can just be filled with zeroes.  We see that 0 is already the mode.

In [16]:
(reduced_df['IHE with >5000 full-time enrollment']==0).value_counts() 

True     357120
False     48559
Name: IHE with >5000 full-time enrollment, dtype: int64

We check the enrollment numbers for the counties and see that those counties which have an enrollment number greater than 5000, have non-null values in IHE > 5000, so where there are nulls, there are less than 5000 full-time-enrolled students.

In [17]:
temp_df = counties_df[(counties_df['IHE Full-time enrollment'] > 5000)]
temp_df[('IHE with >5000 full-time enrollment')].isna().sum()

0

result is 0 so Nulls just mean 0

In [18]:
column_name = 'IHE with >5000 full-time enrollment'
reduced_df[column_name] = reduced_df[column_name].fillna(0)

Below we bin the IHE > 5000 number into 4 groups, rather than treating it like 11 different categories.

We tried one-hot-encoding all the categories; however the diagnostic models returned terrible results, much like the week 2 citibike lab.  So for this round of cleaning, we chose to skip breaking up everything into categories.

In [19]:
reduced_df[column_name].value_counts()
reduced_df[f'{column_name}_cat'] = reduced_df[column_name].map({0:0, 1:1, 2:1, 3:1, 4:2, 5:2, 6:2, 7:2,
                                                               8:3, 9:3, 10:3, 11:3})

In [20]:
reduced_df[column_name+'_cat'].value_counts()

0    363519
1     40920
2       992
3       248
Name: IHE with >5000 full-time enrollment_cat, dtype: int64

In [21]:
reduced_df.head()

,County,FIPS code,County type,CBSA type,State Abbreviation,Population as a percent of state,Population as a percent of national population,IHE with >5000 full-time enrollment,IHE Full-time enrollment as a percent of the population,Cases as a percent of national total - last 7 days,Cases - % change,Deaths - % change,Cases as a percent of national total - previous 7 days,Rapid rise (last 14 days),Number of days of downward case trajectory,Area of Concern Category,Rapid Riser Category,Community Transmission Level - last 7 days,Community Transmission Level - previous 7 days,Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - last 7 days,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting),Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting),Testing latency - absolute change,% tests resulted in 3 or fewer days - absolute change,Confirmed COVID-19 admissions - last 7 days,Confirmed COVID-19 admissions per 100 inpatient beds - last 7 days,Suspected COVID-19 admissions - last 7 days,Suspected COVID-19 admissions per 100 inpatient beds - last 7 days,% inpatient beds occupied,% inpatient beds occupied by COVID-19 patient,% staffed adult ICU beds occupied,% staffed adult ICU beds occupied by COVID-19 patient,% ventilators in use,% ventilators in use by COVID-19 patient,Confirmed COVID-19 admissions per 100 inpatient beds - percent increase,Suspected COVID-19 admissions per 100 inpatient beds - percent increase,% inpatient beds occupied - absolute change,% inpatient beds occupied by COVID-19 patient - absolute change,% staffed adult ICU beds occupied - absolute change,% staffed adult ICU beds occupied by COVID-19 patient - absolute change,% ventilators in use - absolute change,% ventilators in use by COVID-19 patient - absolute change,% hospital CCNs reporting any utilization data at least once - last 7 days,% hospital CCNs reporting all utilization data at least once - last 7 days,% hospital CCNs ever reporting utilization data,% hospital CCNs reporting COVID-19 admissions at least once - last 7 days,People who are fully vaccinated as % of total population,People who are fully vaccinated as % of population - ages 65+,% Uninsured,% In Poverty,% Over Age 65,Average household size,% Non-Hispanic Black,% Hispanic,% Non-Hispanic Native American / Alaskan Native,% Non-Hispanic Asian,SVI score,CCVI score,Forecasted case trajectory,File Date,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting).1,Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting).1,Testing latency - absolute change.1,% tests resulted in 3 or fewer days - absolute change.1,Viral (RT-PCR) lab test positivity rate - 15-21 days ago (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - 15-21 days ago,% Native American / Alaskan Native,% Asian,IHE with >5000 full-time enrollment_cat
0,"Unallocated, MI",26000,NaN,NaN,MI,NaN,NaN,0.0,NaN,0.0602,533.84,160.00,0.000104,NaN,NaN,NaN,NaN,NaN,NaN,0.032,NaN,-0.014,-0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,2021-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,"Miami-Dade County, FL",12086,Large central metro,Metropolitan,FL,0.1265,0.0082,3.0,0.0222,0.0200,-0.06,-0.27,0.019506,0.0,0.0,SustainedHotspot,NaN,High,High,0.112,NaN,-0.002,-0.06,NaN,NaN,573.0,9.0,155.0,3.0,0.84,0.09,0.80,0.15,0.28,0.05,-0.02,-0.07,0.01,0.00,0.01,0.01,0.00,0.00,0.956522,0.913043,1.0,0.956522,0.277,0.675,0.181367,0.180398,0.166587,3.07,0.153529,0.694297,0.000819,0.015059,0.8175,0.814390,Uncertain,2021-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Cook County, IL",17031,Large central metro,Metropolitan,IL,0.4064,0.0155,7.0,0.0263,0.0173,-0.15,-0.13,0.018680

### Changing Rapid Riser Literal to Category and Fill Nulls

We start by replacing null with 'Unknown' so we can see how things sum up in value counts

In [22]:
reduced_df['Rapid Riser Category'].fillna('Unknown', inplace=True)

In [23]:
reduced_df['Rapid Riser Category'].value_counts()

Unknown       374878
Recent         25231
Continuing      4214
New             1356
Name: Rapid Riser Category, dtype: int64

Above we see that the new column only has a few weeks whereas the recents have a lot more entries.  In terms of sorting the categories, it appears that the counties begin with New, due to the low counts.  Recent would be expected next, with Continuing after that.  That said, there is not a clear separate category to indicate when a County has either stopped rising or never started a rapid rise to begin with.  All that is left are nulls, which appears to cover that unknown area.

In [24]:
reduced_df[(reduced_df['Rapid Riser Category'] == 'New')][['County']].value_counts()

County               
Cascade County, MT       3
Albemarle County, VA     3
Story County, IA         3
Orange County, TX        3
Montgomery County, KY    3
                        ..
Grundy County, IL        1
Gunnison County, CO      1
Gurabo Municipio, PR     1
Hale County, TX          1
Zavala County, TX        1
Length: 1156, dtype: int64

Given what was observed, the rapid rise categories are given ordinals in the mapped order.

In [25]:
rapid_rise_col_name = 'Rapid Riser Category'
# fillna (0) -> this was mapped first to Unknown
rapid_rise_col_map = {'Unknown':0, 'New': 1, 'Recent': 2, 'Continuing': 3}
reduced_df[f'{rapid_rise_col_name}_cat'] = reduced_df[rapid_rise_col_name].map(rapid_rise_col_map)

In [26]:
reduced_df[f'{rapid_rise_col_name}_cat']

0         0
1         0
2         0
3         2
4         0
         ..
405674    0
405675    0
405676    0
405677    0
405678    0
Name: Rapid Riser Category_cat, Length: 405679, dtype: int64

In [27]:
reduced_df['Forecasted case trajectory'].value_counts()

Uncertain            364626
Likely Decreasing     40717
Likely Increasing       336
Name: Forecasted case trajectory, dtype: int64

In [28]:
target_col_name = 'Forecasted case trajectory'
target_col_map = {'Likely Decreasing':0, 'Uncertain': 1, 'Likely Increasing': 2}
reduced_df[f'{target_col_name}_cat'] = reduced_df[target_col_name].map(target_col_map)
reduced_df[f'{target_col_name}_cat'].value_counts()

1    364626
0     40717
2       336
Name: Forecasted case trajectory_cat, dtype: int64

In [29]:
reduced_df.shape

(405679, 72)

In [30]:
# Cases - % change 	Deaths - % change
reduced_df['Cases - % change'].isnull().sum()

14722

In [31]:
zero_fillna_columns = ['Cases - % change','Deaths - % change']
death_change_df = reduced_df[(reduced_df['Cases - % change'].isna())][['Forecasted case trajectory','Deaths - % change']]

In [32]:
death_change_df[(death_change_df['Deaths - % change'].isna())]['Forecasted case trajectory'].value_counts()

Uncertain            13012
Likely Decreasing       61
Likely Increasing        2
Name: Forecasted case trajectory, dtype: int64

In [33]:
death_change_df[(death_change_df['Deaths - % change'].notnull())]['Forecasted case trajectory'].value_counts()

Uncertain            1612
Likely Decreasing      35
Name: Forecasted case trajectory, dtype: int64

In [34]:
counties_df.shape

(405679, 104)

In [35]:
reduced_df['FIPS code'].max()

72153

In [36]:
def new_key(row):
    part_1 = str(row['FIPS code'])
    part_1 = f'{part_1.zfill(5)}'
    part_2 = str(row['File Date'])
    return f'{part_1}-{part_2:6}'




In [37]:
reduced_df['Area of Concern Category'].value_counts().index

Index(['ModerateBurden', 'SustainedHotspot', 'ModerateBurdenResolving', 'LowBurden', 'Hotspot', 'EmergingHotspot', 'HighBurdenResolving'], dtype='object')

In [38]:
reduced_df['Forecasted case trajectory_cat'].value_counts()

1    364626
0     40717
2       336
Name: Forecasted case trajectory_cat, dtype: int64

In [39]:
reduced_df['Forecasted case trajectory_cat'].value_counts(normalize=True)

1    0.898804
0    0.100368
2    0.000828
Name: Forecasted case trajectory_cat, dtype: float64

In [40]:
reduced_df[(reduced_df['Area of Concern Category']=='EmergingHotspot')]['Forecasted case trajectory'].value_counts()

Uncertain            9334
Likely Decreasing    1088
Name: Forecasted case trajectory, dtype: int64

In [41]:
col_name = 'Area of Concern Category'
col_map = {'ModerateBurdenResolving':0, 'HighBurdenResolving':1, 'SustainedHotspot':2, 'HotSpot':3, 'LowBurden':4, 'EmergingHotspot':5 }
reduced_df[f'{col_name}_cat'] = reduced_df[col_name].map(col_map)

In [42]:
reduced_df['Area of Concern Category'].isna().sum()

16071

In [43]:
new_reduced_df = reduced_df.copy()

In [44]:
new_reduced_df['Area of Concern Category'].fillna('Unknown', inplace=True)

In [45]:
zero_fillna_columns = ['Cases - % change','Deaths - % change', 'Rapid rise (last 14 days)',
                       'Number of days of downward case trajectory']

In [46]:
reduced_df['Community Transmission Level - last 7 days'].fillna('Unknown', inplace=True)

In [47]:
#Community Transmission Level - last 7 days 	Community Transmission Level - previous 7 days
reduced_df['Community Transmission Level - previous 7 days'].fillna('Unknown', inplace=True)

In [48]:
fillna_col_names_string = 'Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting) 	% tests resulted in 3 or fewer days - last 7 days 	Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting) 	Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting) 	Testing latency - absolute change 	% tests resulted in 3 or fewer days - absolute change 	Confirmed COVID-19 admissions - last 7 days 	Confirmed COVID-19 admissions per 100 inpatient beds - last 7 days 	Suspected COVID-19 admissions - last 7 days 	Suspected COVID-19 admissions per 100 inpatient beds - last 7 days 	% inpatient beds occupied 	% inpatient beds occupied by COVID-19 patient 	% staffed adult ICU beds occupied 	% staffed adult ICU beds occupied by COVID-19 patient 	% ventilators in use 	% ventilators in use by COVID-19 patient 	Confirmed COVID-19 admissions per 100 inpatient beds - percent increase 	Suspected COVID-19 admissions per 100 inpatient beds - percent increase 	% inpatient beds occupied - absolute change 	% inpatient beds occupied by COVID-19 patient - absolute change 	% staffed adult ICU beds occupied - absolute change 	% staffed adult ICU beds occupied by COVID-19 patient - absolute change 	% ventilators in use - absolute change 	% ventilators in use by COVID-19 patient - absolute change 	% hospital CCNs reporting any utilization data at least once - last 7 days 	% hospital CCNs ever reporting utilization data 	% hospital CCNs reporting COVID-19 admissions at least once - last 7 days 	People who are fully vaccinated as % of total population 	People who are fully vaccinated as % of population - ages 65+ 	% Uninsured 	% In Poverty 	% Over Age 65 	Average household size 	% Non-Hispanic Black 	% Hispanic 	% Non-Hispanic Native American / Alaskan Native 	% Non-Hispanic Asian 	SVI score 	CCVI score 	Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting).1 	Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting).1 	Testing latency - absolute change.1 	% tests resulted in 3 or fewer days - absolute change.1 	Viral (RT-PCR) lab test positivity rate - 15-21 days ago (may be an underestimate due to delayed reporting) 	% tests resulted in 3 or fewer days - 15-21 days ago 	% Native American / Alaskan Native 	% Asian'
fillna_col_names = fillna_col_names_string.split(' \t')
more_fillna_col_name_string = 'IHE with >5000 full-time enrollment 	IHE Full-time enrollment as a percent of the population 	Cases as a percent of national total - last 7 days 	Cases - % change 	Deaths - % change 	Cases as a percent of national total - previous 7 days 	Rapid rise (last 14 days) 	Number of days of downward case trajectory 	'
more_fillna_col_names = more_fillna_col_name_string.split(' \t')

fillna_col_names.extend(more_fillna_col_names)
fillna_col_names.remove('')
new_reduced_df = reduced_df.copy()
new_reduced_df[fillna_col_names] = reduced_df[fillna_col_names].fillna(0)


In [49]:
new_reduced_df['County type'].fillna('Unknown', inplace=True)

In [50]:
new_reduced_df['CBSA type'].fillna('Unknown', inplace=True)

In [51]:
new_reduced_df.head()

,County,FIPS code,County type,CBSA type,State Abbreviation,Population as a percent of state,Population as a percent of national population,IHE with >5000 full-time enrollment,IHE Full-time enrollment as a percent of the population,Cases as a percent of national total - last 7 days,Cases - % change,Deaths - % change,Cases as a percent of national total - previous 7 days,Rapid rise (last 14 days),Number of days of downward case trajectory,Area of Concern Category,Rapid Riser Category,Community Transmission Level - last 7 days,Community Transmission Level - previous 7 days,Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - last 7 days,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting),Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting),Testing latency - absolute change,% tests resulted in 3 or fewer days - absolute change,Confirmed COVID-19 admissions - last 7 days,Confirmed COVID-19 admissions per 100 inpatient beds - last 7 days,Suspected COVID-19 admissions - last 7 days,Suspected COVID-19 admissions per 100 inpatient beds - last 7 days,% inpatient beds occupied,% inpatient beds occupied by COVID-19 patient,% staffed adult ICU beds occupied,% staffed adult ICU beds occupied by COVID-19 patient,% ventilators in use,% ventilators in use by COVID-19 patient,Confirmed COVID-19 admissions per 100 inpatient beds - percent increase,Suspected COVID-19 admissions per 100 inpatient beds - percent increase,% inpatient beds occupied - absolute change,% inpatient beds occupied by COVID-19 patient - absolute change,% staffed adult ICU beds occupied - absolute change,% staffed adult ICU beds occupied by COVID-19 patient - absolute change,% ventilators in use - absolute change,% ventilators in use by COVID-19 patient - absolute change,% hospital CCNs reporting any utilization data at least once - last 7 days,% hospital CCNs reporting all utilization data at least once - last 7 days,% hospital CCNs ever reporting utilization data,% hospital CCNs reporting COVID-19 admissions at least once - last 7 days,People who are fully vaccinated as % of total population,People who are fully vaccinated as % of population - ages 65+,% Uninsured,% In Poverty,% Over Age 65,Average household size,% Non-Hispanic Black,% Hispanic,% Non-Hispanic Native American / Alaskan Native,% Non-Hispanic Asian,SVI score,CCVI score,Forecasted case trajectory,File Date,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting).1,Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting).1,Testing latency - absolute change.1,% tests resulted in 3 or fewer days - absolute change.1,Viral (RT-PCR) lab test positivity rate - 15-21 days ago (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - 15-21 days ago,% Native American / Alaskan Native,% Asian,IHE with >5000 full-time enrollment_cat,Rapid Riser Category_cat,Forecasted case trajectory_cat,Area of Concern Category_cat
0,"Unallocated, MI",26000,Unknown,Unknown,MI,NaN,NaN,0.0,0.0000,0.0602,533.84,160.00,0.000104,0.0,0.0,NaN,Unknown,Unknown,Unknown,0.032,0.00,-0.014,-0.07,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,NaN,0.0,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,Uncertain,2021-04-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,NaN
1,"Miami-Dade County, FL",12086,Large central metro,Metropolitan,FL,0.1265,0.0082,3.0,0.0222,0.0200,-0.06,-0.27,0.019506,0.0,0.0,SustainedHotspot,Unknown,High,High,0.112,0.00,-0.002,-0.06,0.0,0.00,573.0,9.0,155.0,3.0,0.84,0.09,0.80,0.15,0.28,0.05,-0.02,-0.07,0.01,0.00,0.01,0.01,0.00,0.00,0.956522,0.913043,1.0,0.956522,0.277,0.675,0.181367,0.180398,0.166587,3.07,0.153529,0.694297,0.00081

Here we reset the index with a combined string of county id (FIPS code) and the file date.  This will let us do the diffs on selected columns to calculate changes which **should** get us closer to the trajectory.  Note that this would be expected since these are features that would appear to be related to a decrease in transmission

In [52]:

new_reduced_df['County_File_Date'] = new_reduced_df.apply(new_key, axis=1)
new_reduced_df = new_reduced_df.set_index('County_File_Date').sort_index(ascending=False)

In [53]:
new_reduced_df.head()

,County,FIPS code,County type,CBSA type,State Abbreviation,Population as a percent of state,Population as a percent of national population,IHE with >5000 full-time enrollment,IHE Full-time enrollment as a percent of the population,Cases as a percent of national total - last 7 days,Cases - % change,Deaths - % change,Cases as a percent of national total - previous 7 days,Rapid rise (last 14 days),Number of days of downward case trajectory,Area of Concern Category,Rapid Riser Category,Community Transmission Level - last 7 days,Community Transmission Level - previous 7 days,Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - last 7 days,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting),Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting),Testing latency - absolute change,% tests resulted in 3 or fewer days - absolute change,Confirmed COVID-19 admissions - last 7 days,Confirmed COVID-19 admissions per 100 inpatient beds - last 7 days,Suspected COVID-19 admissions - last 7 days,Suspected COVID-19 admissions per 100 inpatient beds - last 7 days,% inpatient beds occupied,% inpatient beds occupied by COVID-19 patient,% staffed adult ICU beds occupied,% staffed adult ICU beds occupied by COVID-19 patient,% ventilators in use,% ventilators in use by COVID-19 patient,Confirmed COVID-19 admissions per 100 inpatient beds - percent increase,Suspected COVID-19 admissions per 100 inpatient beds - percent increase,% inpatient beds occupied - absolute change,% inpatient beds occupied by COVID-19 patient - absolute change,% staffed adult ICU beds occupied - absolute change,% staffed adult ICU beds occupied by COVID-19 patient - absolute change,% ventilators in use - absolute change,% ventilators in use by COVID-19 patient - absolute change,% hospital CCNs reporting any utilization data at least once - last 7 days,% hospital CCNs reporting all utilization data at least once - last 7 days,% hospital CCNs ever reporting utilization data,% hospital CCNs reporting COVID-19 admissions at least once - last 7 days,People who are fully vaccinated as % of total population,People who are fully vaccinated as % of population - ages 65+,% Uninsured,% In Poverty,% Over Age 65,Average household size,% Non-Hispanic Black,% Hispanic,% Non-Hispanic Native American / Alaskan Native,% Non-Hispanic Asian,SVI score,CCVI score,Forecasted case trajectory,File Date,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting).1,Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting).1,Testing latency - absolute change.1,% tests resulted in 3 or fewer days - absolute change.1,Viral (RT-PCR) lab test positivity rate - 15-21 days ago (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - 15-21 days ago,% Native American / Alaskan Native,% Asian,IHE with >5000 full-time enrollment_cat,Rapid Riser Category_cat,Forecasted case trajectory_cat,Area of Concern Category_cat
County_File_Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
72153-2021-05-20,"Yauco Municipio, PR",72153,Unknown,Metropolitan,PR,0.0105,0.0001,0.0,0.0,0.0000,-0.71,0.0,0.000120,0.0,0.0,NaN,Unknown,Substantial,High,0.085,0.0,-0.112,-0.26,0.0,0.0,1.0,1.0,0.0,0.0,0.45,0.01,0.67,0.0,0.07,0.0,-0.76,0.0,-0.12,-0.02,0.25,0.0,0.00,0.0,1.0,1.0,1.0,1.0,0.324,0.500,0.053156,0.483875,0.232375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Uncertain,2021-05-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,NaN
72153-2021-05-19,"Yauco Municipio, PR",72153,Unknown,Metropolitan,PR,0.0105,0.0001,0.0,0.0,0.0000,-0.69,0.0,0.000121,0.0,0.0,NaN,Unknown,High,High,0.124,0.0,-0.054,-0.32,0.0,0.0,2.0,3.0,0.0,0.0,0.45,0.01,0.63,0.0,0.09,0.0,-0.37,0.0,-0.13,-0.02,0.18,0.0,0.02,0.0,1.0,1.0,1.0,1.0,0.318,0.497,0.053156,0.483875,0.232375,0

In [54]:
new_reduced_df['Area of Concern Category'].fillna('Unknown', inplace=True)

Here we try to get the mean of the case trajectory as it relates to the area of concern category.  Based on the calculated mean, we'll assign the ordinals for those labels in that order.

In [55]:
sortable_df = new_reduced_df.groupby('Area of Concern Category')['Forecasted case trajectory_cat'].mean().to_frame()
sortable_df.sort_values('Forecasted case trajectory_cat')

,Forecasted case trajectory_cat
Area of Concern Category,
HighBurdenResolving,0.785109
SustainedHotspot,0.800437
ModerateBurdenResolving,0.880722
EmergingHotspot,0.895605
Hotspot,0.949473
ModerateBurden,0.964094
LowBurden,0.993708
Unknown,1.000000


In [56]:
new_reduced_df['Area of Concern Category_cat'] = new_reduced_df['Area of Concern Category'].map({'HighBurdenResolving':0, 
                                                                               'SustainedHotspot':1, 
                                                                               'ModerateBurdenResolving':2, 
                                                                               'EmergingHotspot':3, 'Hotspot':4, 
                                                                               'ModerateBurden':5,
                                                                               'LowBurden':6 })

In [57]:

new_reduced_df['% hospital CCNs reporting all utilization data  at least once - last 7 days'].fillna(0, inplace=True)

Check percentage column values to make sure that 1 means 100%

In [58]:
new_reduced_df['% hospital CCNs reporting all utilization data  at least once - last 7 days'].describe()

count    405679.000000
mean          0.704460
std           0.448616
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: % hospital CCNs reporting all utilization data  at least once - last 7 days, dtype: float64

In [59]:
new_reduced_df['Community Transmission Level - last 7 days'].value_counts()

Unknown        209259
High            75672
Substantial     59128
Moderate        47738
Low             13882
Name: Community Transmission Level - last 7 days, dtype: int64

In [60]:
new_reduced_df['Community Transmission Level - last 7 days'].value_counts().index

Index(['Unknown', 'High', 'Substantial', 'Moderate', 'Low'], dtype='object')

In [61]:
cat_col_name = 'Community Transmission Level - last 7 days'
com_trans_last_7_cat = {'High':4, 'Substantial':3, 'Moderate':2, 'Low':1,'Unknown':0}
new_reduced_df['Com Trans Last7_cat'] = new_reduced_df[cat_col_name].map(com_trans_last_7_cat)

In [62]:
#new_reduced_df['CBSA type'].value_counts()
new_reduced_df['CBSA type'].value_counts().index

Index(['Unknown', 'Metropolitan', 'Micropolitan'], dtype='object')

In [63]:
cat_col_name = 'CBSA type'
cbsa_type_cat = {'Metropolitan':0, 'Micropolitan':1, 'Unknown':2}
new_reduced_df['CBSA type_cat'] = new_reduced_df[cat_col_name].map(cbsa_type_cat)

In [64]:
new_reduced_df['County type'].value_counts().index

Index(['Non-core', 'Micropolitan', 'Medium metro', 'Large fringe metro', 'Small metro', 'Unknown', 'Large central metro'], dtype='object')

In [65]:
new_reduced_df.groupby('County type')['Population as a percent of state'].mean().sort_values().index

Index(['Non-core', 'Micropolitan', 'Unknown', 'Small metro', 'Large fringe metro', 'Medium metro', 'Large central metro'], dtype='object', name='County type')

In [66]:
 # Sorting County type by population percentage to figure out cat number
county_type_cat = {'Non-core':0, 'Micropolitan':1, 'Unknown':2, 'Small metro':3, 
'Large fringe metro':4, 'Medium metro':5, 'Large central metro':6}

cat_col_name = 'County type'

new_reduced_df['County type_cat'] = new_reduced_df[cat_col_name].map(county_type_cat)

In [67]:
fillna_columns = ['Population as a percent of state','Population as a percent of national population']
for col in fillna_columns:
    new_reduced_df[col].fillna(0, inplace=True)


Cases as a percent of national total - last 7 days
Area of Concern Category
People who are fully vaccinated as % of total population
People who are fully vaccinated as % of population - ages 65+

The following columns appear to be good candidates for diffing previous date values.  

Note that the dataset has the rows in order of county (FIPS id) + file date values.  For almost every county, the row with the earliest recorded data will be adjacent to another county's row of their latest data

Suppose we have:

| FIPS ID | File Date | Column to be diffed |
|---|---|---|
| 05 | 2021-04-30  | National Total Cases |
| 05 | 2021-01-02  | National Total Cases |
| 04 | 2021-04-30  | National Total Cases |

We see that the row for FIPS id 05 for 2021-01-02 is right above FIPS id 04 for 2021-04-30.  When the rows get diffed, the *pandas diff* for (id 05)-2021-01-02 will subtract the data from (id 04), which we don't want.  So we'll need to zero that out.

In [68]:
new_reduced_df['cases_national_total_change'] = new_reduced_df['Cases as a percent of national total - last 7 days']
new_reduced_df['total_full_vac_%_change'] = new_reduced_df['People who are fully vaccinated as % of total population']
new_reduced_df['age_65_full_vac_%_change'] = new_reduced_df['People who are fully vaccinated as % of population - ages 65+']


In [69]:

diff_cols = ['cases_national_total_change', 'total_full_vac_%_change', 'age_65_full_vac_%_change']
for col in diff_cols:
    new_reduced_df[col] = new_reduced_df[col].diff(periods=-1)


In [70]:
new_reduced_df.groupby('FIPS code')['File Date'].agg('min')

FIPS code
1000     2021-01-02
1001     2021-01-02
1003     2021-01-02
1005     2021-01-02
1007     2021-01-02
            ...    
72145    2021-01-02
72147    2021-01-02
72149    2021-01-02
72151    2021-01-02
72153    2021-01-02
Name: File Date, Length: 3272, dtype: object

In [71]:
'''Functions to adjust diffing for when one county row is adjacent to another county's row '''

def reset_min_row(row):
    if row['File Date'] == '2021-01-02':
        return 0
    else:
        return row['cases_national_total_change']
        

def reset_total_vac_min_row(row):
    if row['File Date'] == '2021-01-02':
        return 0
    else:
        return row['total_full_vac_%_change']
        

    
def reset_age65_vac_min_row(row):
    if row['File Date'] == '2021-01-02':
        return 0
    else:
        return row['age_65_full_vac_%_change']
        


In [72]:
# cases_national_total_change 	total_full_vac_%_change 	age_65_full_vac_%_change
new_reduced_df['cases_national_total_change'] = new_reduced_df.apply(reset_min_row, axis=1)
new_reduced_df['total_full_vac_%_change'] = new_reduced_df.apply(reset_total_vac_min_row, axis=1)
new_reduced_df['age_65_full_vac_%_change'] = new_reduced_df.apply(reset_age65_vac_min_row, axis=1)

More clean up.  Change the ordinal column names to be more uniform with lower case, so easier for anyone dealing with the data to see what was engineered.

In [73]:
rename_cols_dict = {'IHE with >5000 full-time enrollment_cat':'ihe_plus_5000_fte_cat',
                    'Rapid Riser Category_cat':'rapid_riser_cat',
                    'Forecasted case trajectory_cat':'forecast_case_traj_cat',
                    'Area of Concern Category_cat':'area_of_concern_cat',
                    'Com Trans Last7_cat':'com_trans_last7_cat',
                    'CBSA type_cat':'cbsa_type_cat',
                    'County type_cat':'county_type_cat'}
new_reduced_df.rename(columns=rename_cols_dict, inplace=True)

Remove rows for counties that aren't in the other datasets that this will be joined with.

* Only counties that belong to states are included
* Remove Puerto Rican Counties
* Remove Counties that have the word unallocated in the name

In [74]:
new_reduced_df = new_reduced_df[(new_reduced_df['County'].map(lambda x: 'Unallocated' not in x))]

In [75]:
new_reduced_df = new_reduced_df[(new_reduced_df['State Abbreviation'] != 'PR')]

## Now we have the final version which can be joined with other project data

In [76]:
new_reduced_df.to_csv('../data/counties_diffed.csv', index=False)